<a href="https://colab.research.google.com/github/redrebbaa/SmartMeterVisualisation/blob/main/Image_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install -q jupyter-dash==0.3.0rc1 dash-bootstrap-components transformers
!pip install -q jupyter kaleido
! pip install -q jupyter orca
import sys
!{sys.executable} -m pip install orca
!{sys.executable} -m pip install imgkit

     |████████████████████████████████| 45 kB 1.6 MB/s 
     |████████████████████████████████| 191 kB 6.9 MB/s 
     |████████████████████████████████| 2.6 MB 9.1 MB/s 
     |████████████████████████████████| 1.1 MB 15.7 MB/s 
     |████████████████████████████████| 3.7 MB 45.1 MB/s 
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |████████████████████████████████| 1.8 MB 45.0 MB/s 
     |████████████████████████████████| 636 kB 53.5 MB/s 
     |████████████████████████████████| 3.3 MB 56.7 MB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 
     |████████████████████████████████| 357 kB 47.4 MB/s 
     |████████████████████████████████| 79.9 MB 49 kB/s 
     |████████████████████████████████| 244 kB 6.5 MB/s 


In [ ]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
from transformers import AutoModelWithLMHead, AutoTokenizer
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import imgkit
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib import pylab
from scipy import stats
import seaborn as sns
import PIL
from PIL import Image
import os
import datetime
import time
import torch
import plotly.express as px
from datetime import date
import re

In [ ]:
app = JupyterDash(__name__)
root = '/content/drive/MyDrive/Smart_Meters/house_'
HeatmapFig = html.Div(dcc.Graph(id='HeatmapFig'))

HouseOptions = []
xAxisOptions=[]
xAxisLabels = ['Minute', 'Hour', 'Day', 'Week']
ApplianceOptions =[]
MultiHouseOptions = ['False', 'True']
SamplingOptions = []
MainOptions = [{'label': 'Aggregate Minus or Appliance Combination Selection', 'value': 'Agg'},
               {'label': 'Do you want the figures of multiple houses?', 'value': 'MultiHouse'},
               {'label': 'Create lots of figures?', 'value': 'MultiHouse'}]
RepeatOptions = [{'label': 'Once', 'value': 1}, {'label': 'Twice', 'value': 2},
                 {'label': 'Five Times', 'value': 5}, {'label': 'Ten Times', 'value': 10},
                 {'label': 'Max', 'value': 'Max'}]
for i in range(5):
  HouseOptions.append({'label': str(i+1), 'value': str(i+1)})
for i in xAxisLabels:
  xAxisOptions.append({'label': i, 'value': i})
MainMenu = html.Div(dcc.Checklist(id='MainMenu', options=MainOptions))
HouseMenu = html.Div(dcc.Dropdown(id='HouseMenu', options=HouseOptions,
                        placeholder = "Choose the houses that you want to view"))
 
ApplianceMenu = html.Div(dcc.Dropdown(id='ApplianceMenu', options=ApplianceOptions,
                        multi = True,
                        placeholder = "Choose the appliances that you want to view"))
 
DateMenu = html.Div(dcc.DatePickerRange(
        id='DateMenu'))

SamplingInput = html.Div(dcc.Input(id="SamplingInput", type="number",
                         placeholder="Input the sampling period in seconds", 
                         min=6, step =6))
 
SamplingMenu = html.Div(dcc.Dropdown(id="SamplingMenu", options=SamplingOptions,
                         placeholder="Input the sampling period in seconds"))

xAxisMenu = html.Div(dcc.Dropdown(id='xAxisMenu', options=xAxisOptions,
                        placeholder = "Choose the length of each block of the x axis"))
DownloadButton = html.Button("Download Figures", id="DownloadButton")
DownloadFile = dcc.Download(id="DownloadFile")
RepeatMenu = html.Div(dcc.Dropdown(id='RepeatMenu', options=RepeatOptions,
                        placeholder = "Choose the number of repeats of this graph in time: "))
FileInput = html.Div(dcc.Input(id ='FileInput', type='text', placeholder= 'Input the save location'))

app.layout = html.Div([HeatmapFig, HouseMenu, DateMenu, ApplianceMenu, SamplingMenu, 
                       xAxisMenu, SamplingInput, MainMenu, DownloadButton, DownloadFile, RepeatMenu, FileInput])

@app.callback(
    Output(component_id='HeatmapFig', component_property='figure'),
    Input(component_id='SamplingMenu', component_property='value'),
    Input(component_id='xAxisMenu', component_property='value'),
    Input(component_id='HouseMenu', component_property='value'),
    Input(component_id='ApplianceMenu', component_property='value'),
    Input(component_id='DateMenu', component_property='start_date'),
    Input(component_id='DateMenu', component_property='end_date'),
    Input(component_id='DateMenu', component_property='max_date_allowed'),
    Input(component_id='SamplingInput', component_property='value'),
    Input(component_id='MainMenu', component_property='value'),
    Input(component_id='RepeatMenu', component_property='value'),
    Input(component_id='FileInput', component_property='value'))
def update_output_div(Period, xAxis, HouseNum, Appliances, StartDate, EndDate, MaxDate, InputPeriod, Menu, Repeats, FileLabel):
  root = '/content/drive/MyDrive/Smart_Meters/house_'
  if type(FileLabel) != str:
    path = '/content/drive/MyDrive/Smart_Meters/Figures/ '
  else:
    path = FileLabel
  if type(Period) != int:
    Period = InputPeriod
  if 'MultiHouse' in Menu:
    MultiHouse = True
  MaxDate = MaxDate.split('-')
  MaxDate = datetime.datetime(int(MaxDate[0]), int(MaxDate[1]), int(MaxDate[2])).timestamp()
  StartDate = StartDate.split('-')
  EndDate = EndDate.split('-')
  TimeRange = [datetime.datetime(int(StartDate[0]), int(StartDate[1]), int(StartDate[2])).timestamp(), 
                datetime.datetime(int(EndDate[0]), int(EndDate[1]), int(EndDate[2])).timestamp()]
  TotalTime = TimeRange[1] - TimeRange[0]
  if xAxis == 'Minute':
    DataSize = (int(TotalTime/60), int(60/Period))
  elif xAxis == 'Hour':
    DataSize = (int(TotalTime/3600), int(3600/Period))
  elif xAxis == 'Day':
    DataSize = (int(TotalTime/86400), int(86400/Period))
  elif xAxis == 'Week':
    DataSize = (int(TotalTime/604800), int(604800/Period))
  heatmap = NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, HouseNum, Appliances, root)
  fig = go.Figure(data=go.Heatmap(z = heatmap))
  if Repeats == 'Max':
    Repeats = int((MaxDate - TimeRange[0])/TotalTime)
  elif type(Repeats) != int:
    Repeats = 1
  if MultiHouse:
    FileLabel = ('The ' + ', '.join(Appliances) + ' from all the houses, sampled every ' + 
                str(Period) + ' with an xAxis of ' + xAxis + ' starting on day ' 
                + str(date.fromtimestamp(TimeRange[0])))
  else:
      FileLabel = ('The ' + ', '.join(Appliances) + ' from house '+ HouseNum + 
                  ', sampled every ' + str(Period) + ' with an xAxis of ' + xAxis + 
                  ' starting on day' + str(date.fromtimestamp(TimeRange[0])))
  if Repeats != 1:
    FileLabel += (' Repeated for ' + str(Repeats) + ' and ending on day ' + 
                  str(date.fromtimestamp(TimeRange[1] + TotalTime * Repeats)))
  else:
    FileLabel += (' and ending on day ' + str(date.fromtimestamp(TimeRange[1])))
  for j in range(Repeats):
    if (not os.path.exists('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FileLabel)):
      os.mkdir('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FileLabel)
    if MultiHouse:
      TimeRange = [TimeRange[0] + TotalTime, TimeRange[1] + TotalTime]
      for i in range(1, 6):
        heatmap = NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, str(i), Appliances, root)
        if np.sum(heatmap) != np.nan:
          fig2 = go.Figure(data=go.Heatmap(z = heatmap))
          fig2.write_html('/content/drive/MyDrive/Smart_Meters/Figures/ ' +  FileLabel +
                          '/HouseNumber' +  str(i) + 'Repeat' + str(j) + '.html')
  return fig
 
@app.callback(
    Output(component_id='ApplianceMenu', component_property='options'),
    Input(component_id='HouseMenu', component_property='value'))
def UpdateApplianceLabels(HouseNum):
  ApplianceOptions = []
  root = '/content/drive/MyDrive/Smart_Meters/house_'
  ApplianceLabels = ((pd.read_table(root + HouseNum + '/labels.dat',sep="\s+",names=['Num','Label'])).to_numpy())[:, 1]
  for i in ApplianceLabels:
    ApplianceOptions.append({'label': i, 'value': i})
  return ApplianceOptions
 
@app.callback(
    Output(component_id='DateMenu', component_property='min_date_allowed'),
    Output(component_id='DateMenu', component_property='max_date_allowed'),
    Output(component_id='DateMenu', component_property='initial_visible_month'),
    Input(component_id='HouseMenu', component_property='value'))
def UpdateDateMenu(HouseNum):
  root = '/content/drive/MyDrive/Smart_Meters/house_'
  File = root + HouseNum + '/ aggregate.dat'
  Data = (pd.read_table(File,sep="\s+",names=['t','PR'])).to_numpy()
  MinDate = date.fromtimestamp(Data[0,0])
  MaxDate = date.fromtimestamp(Data[-1,0])
  initial_visible_month = MinDate
  return MinDate, MaxDate, initial_visible_month
 
@app.callback(
    Output(component_id='SamplingMenu', component_property='options'),
    Input(component_id='xAxisMenu', component_property='value'))
def UpdateSamplingMenu(xAxis):
  SamplingOptions = [{'label': 'Choose a specific value: ', 'value': 'Input'}, {'label': '6 seconds', 'value': 6}, {'label': '1 minute', 'value': 60}]
  if xAxis == "Hour":
    SamplingOptions.append({'label': '5 Minutes', 'value': 300})
    SamplingOptions.append({'label': '10 Minutes', 'value': 600})
    SamplingOptions.append({'label': 'Half an Hour', 'value': 1800})
    SamplingOptions.append({'label': 'An Hour', 'value': 3600})
  elif xAxis == 'Day':
    SamplingOptions.append({'label': '5 Minutes', 'value': 300})
    SamplingOptions.append({'label': '10 Minutes', 'value': 600})
    SamplingOptions.append({'label': 'Half an Hour', 'value': 1800})
    SamplingOptions.append({'label': 'An Hour', 'value': 3600})
    SamplingOptions.append({'label': '12 Hours', 'value': 43200}) 
    SamplingOptions.append({'label': 'A day', 'value': 86400})
  elif xAxis == 'Week':
    SamplingOptions.append({'label': '5 Minutes', 'value': 300})
    SamplingOptions.append({'label': '10 Minutes', 'value': 600})
    SamplingOptions.append({'label': 'Half an Hour', 'value': 1800})
    SamplingOptions.append({'label': 'An Hour', 'value': 3600})
    SamplingOptions.append({'label': '12 Hours', 'value': 43200}) 
    SamplingOptions.append({'label': 'A day', 'value': 86400})
    SamplingOptions.append({'label': 'A week', 'value': 86400})
  return SamplingOptions

@app.callback(
    Output("DownloadFile", "data"),
    Input("DownloadButton", "n_clicks"),
    prevent_initial_call=True,)
def func(n_clicks):
  FigureLabel = ('The ' + ', '.join(Appliances) + ' data from all houses with an xAxis = ' 
            + xAxis + ' and a sampling period of ' + str(Period) + ' starting on day ' 
            + str(TimeRange[0]) + ' and ending on day ' + str(TimeRange[1]))
  return dcc.send_file('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FigureLabel + '/HouseNumber1.html')
if __name__ == "__main__":
  app.run_server(host="127.0.0.1",port=8000, debug = True, mode='external')

In [ ]:
def NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, HouseNum, Appliances, root):
  TotalDataPoints = int(TotalTime/Period)
  Data = np.zeros((int(TotalTime/6), (len(Appliances)+1)), int)
  for i in range(len(Appliances)):
    File = root + HouseNum + '/ ' + Appliances[i] + '.dat'
    AppData = ProcessData(File, TimeRange)
    Data[0:((np.shape(AppData))[0]), [0, i+1]] = AppData[:, [0, 1]]
  heatmap = CreateHeatmap(TotalDataPoints, np.sum((Data[:, 1:]),1), Data[:, 0], TimeRange[0], Period, DataSize)
  return heatmap

In [ ]:
def ProcessData(file, TimeRange):
  data = (pd.read_table(file,sep="\s+",names=['t','PR'])).to_numpy()
  data = data[(data[:,0] >= (TimeRange[0])) & (data[:,0] < (TimeRange[1]))]
  return data

In [ ]:
def CreateHeatmap(TotalDataPoints, DataValues, DataLabels, window_start, Period, DataSize):
    heatmap = np.zeros(TotalDataPoints)
    for i in range(TotalDataPoints):
      heatmap[i] = np.mean(DataValues[(DataLabels >= window_start + i*Period) & (DataLabels < (window_start) +((i+6)*Period))])
    heatmap = np.transpose(heatmap.reshape(DataSize))
    heatmap = np.repeat(heatmap, Period, 0)
    return heatmap

In [ ]:
Period = 600
xAxis = 'Day' 
HouseNum = '1'
Appliances = ['aggregate'] 
StartDate = "2013-03-10"
EndDate = "2013-03-15"
MaxDate = "2013-04-08"
InputPeriod = 600
Menu = 'MultiHouse'
Repeats = 2
root = '/content/drive/MyDrive/Smart_Meters/house_'
if type(Period) != int:
  Period = InputPeriod
if 'MultiHouse' in Menu:
  MultiHouse = True
MaxDate = MaxDate.split('-')
MaxDate = datetime.datetime(int(MaxDate[0]), int(MaxDate[1]), int(MaxDate[2])).timestamp()
StartDate = StartDate.split('-')
EndDate = EndDate.split('-')
TimeRange = [datetime.datetime(int(StartDate[0]), int(StartDate[1]), int(StartDate[2])).timestamp(), 
              datetime.datetime(int(EndDate[0]), int(EndDate[1]), int(EndDate[2])).timestamp()]
TotalTime = TimeRange[1] - TimeRange[0]
if xAxis == 'Minute':
  DataSize = (int(TotalTime/60), int(60/Period))
elif xAxis == 'Hour':
  DataSize = (int(TotalTime/3600), int(3600/Period))
elif xAxis == 'Day':
  DataSize = (int(TotalTime/86400), int(86400/Period))
elif xAxis == 'Week':
  DataSize = (int(TotalTime/604800), int(604800/Period))
heatmap = NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, HouseNum, Appliances, root)
fig = go.Figure(data=go.Heatmap(z = heatmap))
fig.write_image("/content/drive/MyDrive/Smart_Meters/Figures/fig1.jpeg")
"""
if Repeats == 'Max':
  Repeats = int((MaxDate - TimeRange[0])/TotalTime)
elif type(Repeats) != int:
  Repeats = 1
if MultiHouse:
  FileLabel = ('The ' + ', '.join(Appliances) + ' from all the houses, sampled every ' + 
               str(Period) + ' with an xAxis of ' + xAxis + ' starting on day ' 
               + str(date.fromtimestamp(TimeRange[0])))
else:
    FileLabel = ('The ' + ', '.join(Appliances) + ' from house '+ HouseNum + 
                 ', sampled every ' + str(Period) + ' with an xAxis of ' + xAxis + 
                 ' starting on day' + str(date.fromtimestamp(TimeRange[0])))
if Repeats != 1:
  FileLabel += (' Repeated for ' + str(Repeats) + ' and ending on day ' + 
                str(date.fromtimestamp(TimeRange[1] + TotalTime * Repeats)))
else:
  FileLabel += (' and ending on day ' + str(date.fromtimestamp(TimeRange[1])))
for j in range(Repeats):
  if (not os.path.exists('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FileLabel)):
    os.mkdir('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FileLabel)
  if MultiHouse:
    TimeRange = [TimeRange[0] + TotalTime, TimeRange[1] + TotalTime]
    for i in range(1, 6):
      if np.sum(heatmap) != np.nan:
        fig2 = go.Figure(data=go.Heatmap(z = NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, str(i), Appliances, root)))
        fig2.write_html('/content/drive/MyDrive/Smart_Meters/Figures/ ' +  FileLabel +
                        '/HouseNumber' +  str(i) + 'Repeat' + str(j) + '.html')
"""

In [ ]:
TimeRange = [1402503695, 1402507295]
Period = 60
xAxis = 'hour'
HouseNum = '1'
Appliances = ['aggregate']
Menu = ['MultiHouse']
root = '/content/drive/MyDrive/Smart_Meters/house_'
DataSize = (int(TotalTime/3600), int(3600/Period))
TotalTime = TimeRange[1] - TimeRange[0]

heatmap = NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, HouseNum, Appliances, root)
fig = go.Figure(data=go.Heatmap(z = heatmap))
FigureLabel = ('The ' + ', '.join(Appliances) + ' data from house number '
+ HouseNum + ' for ' + xAxis + ' starting on day ' + str(TimeRange[0]) + 
' and ending on day ' + str(TimeRange[1]) )
fig.show()
fig.write_html('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FigureLabel + '.html')
if 'MultiHouse' in Menu:
  FigureLabel = ('The ' + ', '.join(Appliances) + ' data from all houses with an xAxis = ' 
          + xAxis + ' and a sampling period of ' + str(Period) + ' starting on day ' 
          + str(TimeRange[0]) + ' and ending on day ' + str(TimeRange[1]))
  if (not os.path.exists('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FigureLabel)):
    os.mkdir('/content/drive/MyDrive/Smart_Meters/Figures/ ' + FigureLabel)
  for i in range(1, 6):
    heatmap = NILMHeatmap(TimeRange, Period, TotalTime, DataSize, xAxis, str(i), Appliances, root)
    if np.sum(heatmap) != np.nan:
      fig2 = go.Figure(data=go.Heatmap(z = heatmap))
      fig2.write_html('/content/drive/MyDrive/Smart_Meters/Figures/ ' +  FigureLabel + '/HouseNumber' +  str(i)+ '.html')

In [ ]:
start = [1352500095, 1352503695]
Period = 60
TotalTime = start[1] - start[0]
DataSize = (int(TotalTime/60), int(60/Period))
DataLength = 'hour'
HouseNum = '1'
Appliances = ['aggregate']
root = '/content/drive/MyDrive/Smart_Meters/house_'
heatmap = NILMHeatmap(start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, root)
fig = go.Figure(data=go.Heatmap(z = heatmap))
fig.show()
fig.write_html("/content/drive/MyDrive/Smart_Meters/Figures/WORKPLEASE.html")

In [ ]:
main()

Input the house number between 1 and 5: 3
Would you like to input data from multiple appliances? Yes/No Yes
How many appliances would you like to input? 2
Choose the appliance name from the list: aggregate, kettle, electric_heater, laptop, projector: Kettle
Choose the appliance name from the list: aggregate, kettle, electric_heater, laptop, projector: Laptop
Choose the appliance name from the list: aggregate, kettle, electric_heater, laptop, projector: laptop
Choose the appliance name from the list: aggregate, kettle, electric_heater, laptop, projector: ketlle
Choose the appliance name from the list: aggregate, kettle, electric_heater, laptop, projector: kettle
Would you like to compare across multiple houses? Yes/No Yes
Input the sampling period: 60
Choose the length of time: Hour, Day, Week, Month, Year  :   Day
Input the start day: 5


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



In [ ]:
def main():
  root = '/content/drive/MyDrive/Smart_Meters/house_'
  start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, MultiHouse = UserInput(root)
  if MultiHouse:
    for i in HouseNum:
      NILMHeatmap(start, Period, TotalTime, DataSize, DataLength, i, Appliances, root)
  else:
    NILMHeatmap(start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, root)

In [ ]:
def UserInput(root):
  HouseNum = None
  Period = 1
  DataLength = 'z'
  TotalTime = 0
  start = [0, 0]
  Appliances = [None]
  NumAppliances = 1
 
  while HouseNum not in ['1', '2', '3', '4', '5']:
    HouseNum = str(input('Input the house number between 1 and 5: '))
  MultiAppliance = input('Would you like to input data from multiple appliances? Yes/No ')
  if MultiAppliance in 'Yes':
    MultiAppliance = True
    NumAppliances = int(input('How many appliances would you like to input? '))
    Appliances = [None] * NumAppliances
  
  ApplianceLabels = ((pd.read_table(root + HouseNum + '/labels.dat',sep="\s+",names=['Num','Label'])).to_numpy())[:, 1]
  for i in range(NumAppliances):
    while Appliances[i] not in ApplianceLabels:
      Appliances[i] = str(input('Choose the appliance name from the list: ' + ', '.join(ApplianceLabels) + ': '))
  
  MultiHouse = input('Would you like to compare across multiple houses? Yes/No ')
  if MultiHouse in 'Yes':
      MultiHouse = True
      HouseNum = [HouseNum]
      for i in range(5):
        ApplianceLabels = ((pd.read_table(root + str(i+1) + '/labels.dat',sep="\s+",names=['Num','Label'])).to_numpy())[:, 1]
        if (set(Appliances).issubset(ApplianceLabels)) and (str(i+1) not in HouseNum):
          HouseNum.append(str(i+1))
  else:
    MultiHouse = False
  
  while Period % 6 != 0:
    Period = int(input('Input the sampling period: '))
  while TotalTime == 0:
    if DataLength in 'Hour':
      TotalTime = 3600
      DataSize = (60, int(60/Period))
      DataLength = 'hour'
    elif DataLength in 'Day':
      TotalTime = 86400
      DataSize = (24, int(3600/Period))
      DataLength = 'day'
    elif DataLength in 'Week':
      TotalTime = 604800
      DataSize = (7, int(86400/Period))
      DataLength = 'week'
    elif DataLength in 'Month':
      TotalTime = 2678400
      DataSize = (31, int(86400/Period))
      DataLength = 'month'
    elif DataLength in 'Year':
      TotalTime = 31536000
      DataSize = (365, int(86400/Period))
      DataLength = 'year'
    else:
      DataLength = input('Choose the length of time: Hour, Day, Week, Month, Year  :   ')
  start[0] = (int(input('Input the start day: ')))
  if DataLength == 'hour':
    start[1] = (int(input('Input the start hour: ')))
    DataLength = 'an ' + DataLength
  else:
    DataLength = 'a ' + DataLength
  return start, Period, TotalTime, DataSize, DataLength, HouseNum, Appliances, MultiHouse